In [84]:
! rm -r /content/1640708290

In [1]:
!pip install keras-tuner --upgrade

In [33]:
import matplotlib.pyplot as plt 
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, Activation, MaxPooling2D, Flatten
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [34]:
import time 

LOG_DIR = f'{int(time.time())}'

In [35]:
# load fashion mnist dataset 
(training_images, training_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

# reshape 
training_images = training_images.reshape(training_images.shape[0], 28, 28, 1)

test_images = test_images.reshape(test_images.shape[0], 28,28,1)
print(training_images.shape)

(60000, 28, 28, 1)


In [85]:
def build_model(hp): # random search passes this hyperparameter() object 
  model = keras.models.Sequential()

  model.add(Conv2D(hp.Int('input_units', min_value=32, max_value=256, step=32), (3, 3), input_shape=training_images.shape[1:]))
  model.add(Activation(hp.Choice("activation", ["relu", 'LeakyReLU', "tanh", 'softplus', 'softsign'])))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  for i in range(hp.Int('n_layers', 1, 4)):
    model.add(Conv2D(hp.Int(f'conv_{i}_units', min_value=32, max_value=256, step=32), (3, 3)))
    model.add(Activation(hp.Choice("activation", ["relu", 'LeakyReLU', "tanh", 'softplus', 'softsign'])))
    model.add(Conv2D(hp.Int(f'conv_{i}_units', min_value=32, max_value=256, step=32), (3, 3)))


  model.add(Flatten())

  model.add(Dense(10))
  model.add(Activation('softmax'))

  learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', 
                metrics=['accuracy'])
  
  return model

In [86]:
tuner = kt.RandomSearch(
    build_model, 
    objective = 'val_accuracy', 
    max_trials = 3, 
    executions_per_trial = 1, 
    directory = LOG_DIR
)

In [87]:
tuner.search(x=training_images, 
             y=training_labels, 
             epochs=5, 
             batch_size=64, 
             validation_data=(test_images, test_labels))

Trial 3 Complete [00h 01m 22s]
val_accuracy: 0.8902000188827515

Best val_accuracy So Far: 0.8970999717712402
Total elapsed time: 00h 03m 52s
INFO:tensorflow:Oracle triggered exit


In [88]:
import pickle 

with open(f'tuner_{int(time.time())}.pkl', 'wb') as f: 
  pickle.dump(tuner, f)


In [89]:

tuner = pickle.load(open('/content/tuner_1640713828.pkl', 'rb'))

print(tuner.get_best_hyperparameters()[0].values)


{'input_units': 160, 'activation': 'softsign', 'n_layers': 3, 'conv_0_units': 64, 'lr': 0.00042971886667723735, 'conv_1_units': 32, 'conv_2_units': 32}


In [90]:
print(tuner.results_summary())

Results summary
Results in 1640708290/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
input_units: 160
activation: softsign
n_layers: 3
conv_0_units: 64
lr: 0.00042971886667723735
conv_1_units: 32
conv_2_units: 32
Score: 0.8970999717712402
Trial summary
Hyperparameters:
input_units: 128
activation: tanh
n_layers: 1
conv_0_units: 128
lr: 0.00048078854019110225
conv_1_units: 128
conv_2_units: 96
Score: 0.8902000188827515
Trial summary
Hyperparameters:
input_units: 96
activation: LeakyReLU
n_layers: 1
conv_0_units: 128
lr: 0.00012848373778229555
conv_1_units: 224
conv_2_units: 192
Score: 0.8896999955177307
None


In [91]:
print(tuner.get_best_models()[0].summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 160)       1600      
                                                                 
 activation (Activation)     (None, 26, 26, 160)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 160)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        92224     
                                                                 
 activation_1 (Activation)   (None, 11, 11, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 64)          36928     
                                                        